[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mahmouddraz/xai/blob/main/notebooks/homework/grad_cam/GradCAM_Exercise.ipynb)

# Review 
**CAM**: uses the weights from the golabe average pooling layer to weight the activation of the last convoutional layer. cam_values = np.dot(weights, conv_values)

**Saliency** uses calculates the gradinets of the loss and overlay it on the orginal image 
saleincy = grad(loss, conv_values)

**GradCAM** uses the gradient to wieght the activations of the last convultional layer (it combines CAM and Salieny). The benfits is: more accurate allocation of the  

grad_cam = np.dot(grad(loss, conv), conv_values)

# GradCAM

## Imports

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf 
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds 
import tensorflow_hub as hub 
from tensorflow import keras
import tensorflow.keras.backend as k 

import cv2 
import numpy as np 
import seaborn as sn 
import pandas as pd 

import matplotlib.pyplot as plt

## Load dataset (CATS_VS_DOGS)

In [ ]:
# chnage the source of the datasets, the one in new version of tensorflow is not wokring
setattr(tfds.image_classification.cats_vs_dogs, '_URL', 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip')

In [ ]:
# do not show the prcess and progress bar while loading
tfds.disable_progress_bar()

splits = ['train[:80%]', 'train[80%:90%]', 'train[90%:]']

# load the dataset given the splits defined above
dataset, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True, split=splits)

(train_examples, validation_examples, test_examples) = dataset

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [ ]:
# resizes the image and normalizes the pixel values
def normalize(image, label):
  image = tf.image.resize(image, (224, 224)) / 255.0
  return  image, label

# prepare batches
train_batches = train_examples.shuffle(num_examples // 4).map(normalize).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
validation_batches = validation_examples.map(normalize).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
# for testing, we make a batch size of 1 image every batch
test_batches = test_examples.map(normalize).batch(1)

## Import model

In [ ]:
# load a pretrained model if the training is taking very long time 
model = keras.models.load_model('/content/drive/MyDrive/XAI workshop/Models/cnn_grad_cam.h5')

###  Activation Model

In [ ]:
# detrmine the layers you would like to track their activation
############################### CODE HERE ###############################

# create a another model from the original that returns the activation sof these 18  layers

############################### CODE HERE ###############################

### Gradients model

In [ ]:
#get the layer names
layer_names = []
for layer in outputs: 
    layer_names.append(layer.name.split('/')[0])

In [ ]:
# This model adjust the output to return the gradients in a certain layer giving its name

############################### CODE HERE ###############################

### Prepare a testing image

In [ ]:
# select a random image for the testing dataset
for img, label in test_batches.shuffle(1000).take(1): 
  sample_image = img[0]
  sample_label = label[0]

# expand its dimentions to match the input shape of the model that will predict its label, get its activation and calculate the gradients
sample_image_processed = np.expand_dims(sample_image, axis=0)

### Get the prediction of the image

In [ ]:
# by applyging the argmax function on the output of the original model
pred_label = np.argmax(model.predict(sample_image_processed), axis=-1)[0]
pred_label

1

### Get the activations

In [ ]:
# call the activations model to get the activation tensors in al 16 layers 

############################### CODE HERE ###############################

# stadrize the vlaue by taking the mean and divding by stadard divation
sample_activation = activations[0][0,:,:,16]
sample_activation-=sample_activation.mean()
sample_activation/=sample_activation.std()

### Calculate the loss and the gradients of the loss

In [ ]:
with tf.GradientTape() as tap:

    #apply the the grad model to the the image you want to explain 
    ############################### CODE HERE ###############################
    
    # asks GradnientTap to recod the values of the conv layer  (you need it for the head map)
    ############################### CODE HERE ###############################

    #get the predicted label from the prediction results 
    ############################### CODE HERE ###############################

    # convert the the ture label to float32 
    ############################### CODE HERE ###############################

    # # You want to calcuata the gradient of the loss over the layer values
    # # loss = accual_label * log(pred_label)
    # # recall: cross entropy is measures the diffrence between the probability of two random varaibles. 
    # # in our case the these variabels are the accual_label and the pred_label. 
    
    # # claculate the losss : Cross  entropy 
    loss =  -1 * (actual_label * tf.math.log(pred_label + 0.00001) + (1 - actual_label) * tf.math.log(1- pred_label + 0.00001))


In [ ]:
# calcualte the gradient of of the loss (None, 224, 224, 512)
############################### CODE HERE ###############################

# Take the mean of all feature frams, shape([512])
############################### CODE HERE ###############################

# squeeze the tensor, outshape an tensor with (512)
############################### CODE HERE ###############################

# take out the the numpy array 
############################### CODE HERE ###############################

### The fun part

In [ ]:
np.__version__

'1.21.6'

In [ ]:
#weight the each feature map in the activations (224, 224, 512) or the output values of the conv layer by the gradients (512)

# We weight all feature map in in the last layer by  by the grad 
### that is why it is callsed GRAD-weighted ## 
############################### CODE HERE ###############################

ValueError: ignored

### Calculate the heatmap

In [ ]:
############################### CODE HERE ###############################

### Plot

In [ ]:
f,ax = plt.subplots(2,2, figsize=(15,8))

ax[0,0].imshow(sample_image)
ax[0,0].set_title(f"True label: {sample_label} \n Predicted label: {pred_label}")
ax[0,0].axis('off')

ax[0,1].imshow(sample_activation)
ax[0,1].set_title("Random feature map")
ax[0,1].axis('off')

ax[1,0].imshow(heatmap)
ax[1,0].set_title("Class Activation Map")
ax[1,0].axis('off')

ax[1,1].imshow(super_imposed_image)
ax[1,1].set_title("Activation map superimposed")
ax[1,1].axis('off')
plt.tight_layout()
plt.show()

### Task


- Select the output of other conv layers and compare the graphs
- Track the shape changes of the activations, gradients and image throughout the process and justify the changes

## In case you do nto have a pretrained model

#### Run this part of the note book afete loading the datasets if you do not have a pretrain mdoel. 

In [ ]:
from tensorflow.keras.applications import vgg16

In [ ]:
base_model = vgg16.VGG16(input_shape=input_shape, weights='imagenet', include_top=False)

In [ ]:
output = keras.layers.GlobalAveragePooling2D()(base_model.output) 
output = keras.layers.Dense(2, activation='softmax', )(output) 

In [ ]:
model = Model(inputs=base_model.input, outputs=output)

# change the model the make the output you want to visualize 
visaulization_model = Model(inputs=model.input, outputs=outputs)

In [ ]:
for layer in base_model.layers[:-4]:
   layer.trainable=False 

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(0.001)

In [ ]:
from tensorflow.python.eager.monitoring import Metric
model.compile(
    optimizer=optimizer, 
    loss='sparse_categorical_crossentropy', 
    metrics='accuracy'
    )

In [ ]:
model.summary()